In [37]:
def compute_strains_3stations(lons, lats):

    from numpy.linalg import inv
    
    xcentroid = np.mean(lons)
    ycentroid = np.mean(lats)
        
    dE1 = (lons[0] - xcentroid) * 111.0 * np.cos(np.deg2rad(ycentroid));
    dE2 = (lons[1] - xcentroid) * 111.0 * np.cos(np.deg2rad(ycentroid));
    dE3 = (lons[2] - xcentroid) * 111.0 * np.cos(np.deg2rad(ycentroid));
    dN1 = (lats[0] - ycentroid) * 111.0;
    dN2 = (lats[1] - ycentroid) * 111.0;
    dN3 = (lats[2] - ycentroid) * 111.0;
    
    Design_Matrix = np.array(
            [[1, 0, dE1, dN1, 0, 0], [0, 1, 0, 0, dE1, dN1], [1, 0, dE2, dN2, 0, 0], [0, 1, 0, 0, dE2, dN2],
             [1, 0, dE3, dN3, 0, 0], [0, 1, 0, 0, dE3, dN3]]);

    # Invert to get the components of the velocity gradient tensor.
    DMinv = inv(Design_Matrix);
    
    def inner_function(uxs, uys):
        (VE1, VE2, VE3) = uxs
        (VN1, VN2, VN3) = uys
    
        obs_vel = np.array([[VE1], [VN1], [VE2], [VN2], [VE3], [VN3]]);

        vel_grad = np.dot(DMinv, obs_vel);  # this is the money step.
    
        dudx = vel_grad[2][0];
        dudy = vel_grad[3][0];
        dvdx = vel_grad[4][0];
        dvdy = vel_grad[5][0];
    
        exx = dudx;
        exy = (0.5 * (dvdx + dudy));
        eyy = dvdy;
        rot = (0.5 * (dvdx - dudy));
        
        return [exx, exy, eyy, rot];
    
    return inner_function


In [23]:
import csv
from dateutil.parser import isoparse
import numpy as np

def read_gps_data(filename):
    date = []
    ux = []
    uy = []
    uz = []
    sig_ux = []
    sig_uy = []
    sig_uz = []

    with open(filename, 'r') as file:
        reader = csv.reader(file, delimiter = ',')
    
        for _ in range(5):
            next(reader)
    
        line = next(reader)[0]
        import re
        match = re.search('Latitude: ([+-]?([0-9]*[.])?[0-9]+)', line)
        if match:
            lat = float(match.group(1))
        match = re.search('Longitude: ([+-]?([0-9]*[.])?[0-9]+)', line)
        if match:
            lon = float(match.group(1))
        for _ in range(3):
            next(reader)
            
        for row in reader:
            date += [isoparse(row[0])]
            ux += [float(row[1])]
            uy += [float(row[2])]
            uz += [float(row[3])]
            sig_ux += [float(row[4])]
            sig_uy += [float(row[5])]
            sig_uz += [float(row[6])]
    
    date = np.array(date, dtype = np.datetime64)
    ux = np.array(ux)
    uy = np.array(uy)
    uz = np.array(uz)
    sig_ux = np.array(sig_ux)
    sig_uy = np.array(sig_uy)
    sig_uz = np.array(sig_uz)
    
    return date, ux, uy, uz, sig_ux, sig_uy, sig_uz, lat, lon

In [27]:
date_knol, ux_knol, uy_knol, uz_knol, _, _, _, lat_knol, lon_knol = read_gps_data('data/GPS/KNOL-cwu-nam14-gpos.csv')
date_p630, ux_p630, uy_p630, uz_p630, _, _, _, lat_p630, lon_p630 = read_gps_data('data/GPS/P630-cwu-nam14-gpos.csv')
date_linc, ux_linc, uy_linc, uz_linc, _, _, _, lat_linc, lon_linc = read_gps_data('data/GPS/LINC-cwu-nam14-gpos.csv')

_, indexes, _ = np.intersect1d(date, dates_m, assume_unique = True, return_indices = True)

/Users/hilley/opt/miniconda2/envs/topoanalysis/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


In [38]:
lats = np.array([lat_knol, lat_p630, lat_linc])
lons = np.array([lon_knol, lon_p630, lon_linc])

strain_computer = compute_strains_3stations(lons, lats)

In [43]:
mean_normal_strain = np.zeros_like(ux_linc)

for index, (ux_1, ux_2, ux_3, uy_1, uy_2, uy_3) in enumerate(zip(ux_knol, ux_p630, ux_linc, uy_knol, uy_p630, uy_linc)):
    [exx, exy, eyy, rot] = strain_computer((ux_1, ux_2, ux_3), (uy_1, uy_2, uy_3))
    mean_normal_strain[index] = (exx + eyy) / 2.0

In [45]:
mean_normal_strain

array([0.14261402, 0.14296931, 0.14405144, ..., 0.        , 0.        ,
       0.        ])